In [50]:
import pandas as pd
from sklearn import *
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [17]:
df = pd.read_csv("/data/kaggle/data_combined_cleaned.csv")
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalesPrice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,...,0,None,None,None,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,...,0,None,None,None,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,...,0,None,None,None,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,...,0,None,None,None,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,...,0,None,None,None,0,12,2008,WD,Normal,250000.0


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 80 columns):
Id               2919 non-null int64
MSSubClass       2919 non-null int64
MSZoning         2919 non-null object
LotFrontage      2919 non-null float64
LotArea          2919 non-null int64
Street           2919 non-null object
Alley            2919 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null int64
OverallCond      2919 non-null int64
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd      2919 non

In [19]:
features = dict(df.dtypes)
features_numeric = [k for k in features 
                    if features[k] in ["int64", "float64"]]
features_categorical = [k for k in features 
                    if features[k] not in ["int64", "float64"]]

len(features_numeric), len(features_categorical)

(38, 42)

In [20]:
# You should always drop the identifier type columns from the features 
df = df.drop(columns=["Id"])

df = df[~df.SalesPrice.isnull()]

In [21]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalesPrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,...,0,None,None,None,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,...,0,None,None,None,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,...,0,None,None,None,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,...,0,None,None,None,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,...,0,None,None,None,0,12,2008,WD,Normal,250000.0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 79 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1460 no

In [24]:
target = "SalesPrice"
X = df.drop(columns=[target]) # Features

y = df[target] # Label/outcome/target

# Log transformation to reduce the impact of the outliers on y values
#y = np.log(y) 

# One hot Encoding - converts the categorical data into numeric
X = pd.get_dummies(X, drop_first=True)
features = X.columns

# X.values returns numpy array for the dataframe
# astype converts each feature to floating point number
X = X.values.astype("float")

# Divide the dataset into training and test
# Model is trained on training set 
# Model is evaluated on the test set
# Model has not seen the test data and assuming test records is a good 
# representation of future, we can get an idea how the model will perform
# in production
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                                , test_size = 0.3, random_state = 1)



pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2
    #                                          , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.LinearRegression())
])


pipe.fit(X_train, y_train) 
# training is happening to build the model
# Essetially, the coefficients/weights/parameters values are calculated

# Doing the prediction
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)


print("mse for test: "
      , metrics.mean_squared_error(y_test, y_test_pred))
print("mse for train: "
      , metrics.mean_squared_error(y_train, y_train_pred))
print("r2 for test: "
      , metrics.r2_score(y_test, y_test_pred))
print("r2 for train: "
      , metrics.r2_score(y_train, y_train_pred))

mse for test:  2.166582758454973e+32
mse for train:  394308817.91726154
r2 for test:  -3.0343009424960066e+22
r2 for train:  0.9337148613558154


In [28]:
est = pipe.steps[-1][-1]
pd.DataFrame({"feature": features
              , "coefficient": est.coef_})

,feature,coefficient
0,MSSubClass,3.252678e+03
1,LotFrontage,2.819351e+02
2,LotArea,8.333517e+03
3,OverallQual,7.471981e+03
4,OverallCond,5.783810e+03
5,YearBuilt,7.910848e+03
6,YearRemodAdd,1.687915e+03
7,MasVnrArea,2.335821e+03
8,BsmtFinSF1,8.752049e+15
9,BsmtFinSF2,3.114624e+15


In [47]:
target = "SalesPrice"
X = df.drop(columns=[target]) # Features

y = df[target] # Label/outcome/target

# Log transformation to reduce the impact of the outliers on y values
#y = np.log(y) 

# One hot Encoding - converts the categorical data into numeric
X = pd.get_dummies(X, drop_first=True)
features = X.columns

# X.values returns numpy array for the dataframe
# astype converts each feature to floating point number
X = X.values.astype("float")

# Divide the dataset into training and test
# Model is trained on training set 
# Model is evaluated on the test set
# Model has not seen the test data and assuming test records is a good 
# representation of future, we can get an idea how the model will perform
# in production
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                                , test_size = 0.3, random_state = 1)



pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2
    #                                          , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha=330.0, max_iter=10000))
])


pipe.fit(X_train, y_train) 
# training is happening to build the model
# Essetially, the coefficients/weights/parameters values are calculated

# Doing the prediction
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)


print("mse for test: "
      , metrics.mean_squared_error(y_test, y_test_pred))
print("mse for train: "
      , metrics.mean_squared_error(y_train, y_train_pred))
print("r2 for test: "
      , metrics.r2_score(y_test, y_test_pred))
print("r2 for train: "
      , metrics.r2_score(y_train, y_train_pred))

mse for test:  955279993.6538922
mse for train:  495046234.04622036
r2 for test:  0.8662129579966458
r2 for train:  0.9167804351108357


In [53]:
est = pipe.steps[-1][-1]
summary = pd.DataFrame({"feature": features
              , "coefficient": est.coef_})
summary["mag"] = np.abs(summary.coefficient)
summary = summary.sort_values("mag", ascending = False)
summary

,feature,coefficient,mag
110,RoofMatl_CompShg,34327.030945,34327.030945
15,GrLivArea,23411.311876,23411.311876
114,RoofMatl_Tar&Grv,23253.070718,23253.070718
116,RoofMatl_WdShngl,22063.091997,22063.091997
115,RoofMatl_WdShake,12439.534009,12439.534009
3,OverallQual,11241.410513,11241.410513
151,ExterQual_TA,-11164.703903,11164.703903
200,KitchenQual_Gd,-9227.039675,9227.039675
150,ExterQual_Gd,-9158.996664,9158.996664
201,KitchenQual_TA,-9107.134963,9107.134963


In [57]:
target = "SalesPrice"
X = df.drop(columns=[target]) # Features

y = df[target] # Label/outcome/target

# Log transformation to reduce the impact of the outliers on y values
y = np.log(y) 

# One hot Encoding - converts the categorical data into numeric
X = pd.get_dummies(X, drop_first=True)
features = X.columns

# X.values returns numpy array for the dataframe
# astype converts each feature to floating point number
X = X.values.astype("float")

# Divide the dataset into training and test
# Model is trained on training set 
# Model is evaluated on the test set
# Model has not seen the test data and assuming test records is a good 
# representation of future, we can get an idea how the model will perform
# in production
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                                , test_size = 0.3, random_state = 1)



pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2
    #                                          , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha=0.01, max_iter=10000))
])


pipe.fit(X_train, y_train) 
# training is happening to build the model
# Essetially, the coefficients/weights/parameters values are calculated

# Doing the prediction
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)


print("mse for test: "
      , metrics.mean_squared_error(y_test, y_test_pred))
print("mse for train: "
      , metrics.mean_squared_error(y_train, y_train_pred))
print("r2 for test: "
      , metrics.r2_score(y_test, y_test_pred))
print("r2 for train: "
      , metrics.r2_score(y_train, y_train_pred))

mse for test:  0.022954852934347732
mse for train:  0.01552703945002047
r2 for test:  0.8785389987905549
r2 for train:  0.8940480673803641


In [74]:
target = "SalesPrice"
X = df.drop(columns=[target]) # Features

y = df[target] # Label/outcome/target

# Log transformation to reduce the impact of the outliers on y values
y = np.log(y) 

# One hot Encoding - converts the categorical data into numeric
X = pd.get_dummies(X, drop_first=True)
features = X.columns

# X.values returns numpy array for the dataframe
# astype converts each feature to floating point number
X = X.values.astype("float")

# Divide the dataset into training and test
# Model is trained on training set 
# Model is evaluated on the test set
# Model has not seen the test data and assuming test records is a good 
# representation of future, we can get an idea how the model will perform
# in production
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                                , test_size = 0.3, random_state = 1)



pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2
    #                                          , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Ridge(alpha=50, max_iter=10000))
])


pipe.fit(X_train, y_train) 
# training is happening to build the model
# Essetially, the coefficients/weights/parameters values are calculated

# Doing the prediction
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)


print("mse for test: "
      , metrics.mean_squared_error(y_test, y_test_pred))
print("mse for train: "
      , metrics.mean_squared_error(y_train, y_train_pred))
print("r2 for test: "
      , metrics.r2_score(y_test, y_test_pred))
print("r2 for train: "
      , metrics.r2_score(y_train, y_train_pred))

mse for test:  0.023064529651755716
mse for train:  0.009139586983626926
r2 for test:  0.8779586664336524
r2 for train:  0.9376341570215264


In [82]:
target = "SalesPrice"
X = df.drop(columns=[target]) # Features

y = df[target] # Label/outcome/target

# Log transformation to reduce the impact of the outliers on y values
y = np.log(y) 

# One hot Encoding - converts the categorical data into numeric
X = pd.get_dummies(X, drop_first=True)
features = X.columns

# X.values returns numpy array for the dataframe
# astype converts each feature to floating point number
X = X.values.astype("float")

# Divide the dataset into training and test
# Model is trained on training set 
# Model is evaluated on the test set
# Model has not seen the test data and assuming test records is a good 
# representation of future, we can get an idea how the model will perform
# in production
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                                , test_size = 0.3, random_state = 123424)



pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2
    #                                          , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.SGDRegressor(alpha=1.0
                                      , l1_ratio=0.4
                                      , eta0 = 0.01
                                      , tol = 1e-5
                                      , learning_rate="invscaling"
                                      , max_iter=10000))
])


pipe.fit(X_train, y_train) 
# training is happening to build the model
# Essetially, the coefficients/weights/parameters values are calculated

# Doing the prediction
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)


print("mse for test: "
      , metrics.mean_squared_error(y_test, y_test_pred))
print("mse for train: "
      , metrics.mean_squared_error(y_train, y_train_pred))
print("r2 for test: "
      , metrics.r2_score(y_test, y_test_pred))
print("r2 for train: "
      , metrics.r2_score(y_train, y_train_pred))

mse for test:  0.02892651770014163
mse for train:  0.018446418926622837
r2 for test:  0.8279565099824034
r2 for train:  0.8815245479380311


In [84]:
scores = model_selection.cross_val_score(pipe, X, y, cv = 5 )
scores

array([0.87178369, 0.80096496, 0.85077457, 0.87206999, 0.80228642])

In [85]:
np.mean(scores)

0.839575924746472

In [88]:
pipe1 = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2
    #                                          , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.SGDRegressor(alpha=0.8
                                      , l1_ratio=0.4
                                      , eta0 = 0.01
                                      , tol = 1e-5
                                      , learning_rate="invscaling"
                                      , max_iter=10000))
])
pipe2 = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2
    #                                          , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.SGDRegressor(alpha=0.9
                                      , l1_ratio=0.4
                                      , eta0 = 0.01
                                      , tol = 1e-5
                                      , learning_rate="invscaling"
                                      , max_iter=10000))
])

cv_score1 = np.mean(model_selection.cross_val_score(pipe1, X, y, cv = 5))
cv_score2 = np.mean(model_selection.cross_val_score(pipe2, X, y, cv = 5))

cv_score1, cv_score2

(0.8321727174335685, 0.8370606482855754)

In [89]:
10 ** np.linspace(-2, 2, 10)

array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])

In [91]:
pipe1 = pipeline.Pipeline([
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.SGDRegressor(alpha=0.8
                                      , l1_ratio=0.4
                                      , eta0 = 0.01
                                      , tol = 1e-5
                                      , learning_rate="invscaling"
                                      , max_iter=10000))
])

param_grid = {
    "est__alpha": 10 ** np.linspace(-2, 2, 10),
    "est__l1_ratio": np.linspace(0, 1, 10)
}
gsearch = model_selection.GridSearchCV(pipe, param_grid
                                       , cv = 5, verbose=True)

gsearch.fit(X, y)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    8.3s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', SGDRegressor(alpha=1.0, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.4,
       learning_rate='invscaling', loss='squared_los...state=None, shuffle=True, tol=1e-05, validation_fraction=0.1,
       verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'est__alpha': array([1.00000e-02, 2.78256e-02, 7.74264e-02, 2.15443e-01, 5.99484e-01,
       1.66810e+00, 4.64159e+00, 1.29155e+01, 3.59381e+01, 1.00000e+02]), 'est__l1_ratio': array([0.     , 0.11111, 0.22222, 0.33333, 0.44444, 0.55556, 0.66667,
       0.77778, 0.88889, 1.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [92]:
gsearch.best_params_

{'est__alpha': 0.5994842503189409, 'est__l1_ratio': 0.8888888888888888}

In [93]:
gsearch.best_score_

0.8577011797028022

In [96]:
y_test_pred = gsearch.predict(X_test)
metrics.r2_score(y_test, y_test_pred)

0.8949297511614906

In [97]:
import pickle

In [98]:
with open("model.pkl", "wb") as f:
    pickle.dump(gsearch, f)

In [99]:
!ls -lh

total 1432
-rw-r--r--  1 abasar  staff   139K Nov 26 12:21 Day 1 - dataframe.ipynb
-rw-r--r--  1 abasar  staff    27K Nov 26 11:00 Day 1 - python.ipynb
-rw-r--r--  1 abasar  staff    24K Nov 27 12:00 Day 1 - regression.ipynb
-rw-r--r--  1 abasar  staff    14K Nov 26 12:55 Day 1 - top 10 movies.ipynb
-rw-r--r--  1 abasar  staff   229K Nov 27 14:35 Day 1 - visualization.ipynb
-rw-r--r--  1 abasar  staff    36K Nov 27 14:39 Day 2 - null treatment.ipynb
-rw-r--r--  1 abasar  staff    66K Nov 27 12:59 Regression - Power Demand Forecast.ipynb
-rw-r--r--  1 abasar  staff    74K Nov 27 16:11 Untitled.ipynb
-rw-r--r--  1 abasar  staff    31K Nov 26 14:55 bitcoing.jpg
-rw-r--r--@ 1 abasar  staff    14K Nov 27 11:58 bitcoing.pdf
-rw-r--r--  1 abasar  staff    40K Nov 27 16:12 model.pkl
-rw-r--r--  1 abasar  staff   497B Nov 26 12:48 top_to_movies.csv
-rw-r--r--  1 abasar  staff   596B Nov 26 12:48 top_to_movies.json


In [101]:
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

In [102]:
model

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', SGDRegressor(alpha=1.0, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.4,
       learning_rate='invscaling', loss='squared_los...state=None, shuffle=True, tol=1e-05, validation_fraction=0.1,
       verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'est__alpha': array([1.00000e-02, 2.78256e-02, 7.74264e-02, 2.15443e-01, 5.99484e-01,
       1.66810e+00, 4.64159e+00, 1.29155e+01, 3.59381e+01, 1.00000e+02]), 'est__l1_ratio': array([0.     , 0.11111, 0.22222, 0.33333, 0.44444, 0.55556, 0.66667,
       0.77778, 0.88889, 1.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [103]:
y_test_pred = model.predict(X_test)
metrics.r2_score(y_test, y_test_pred)

0.8949297511614906